In [1]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
test_dir='test'

test_generator = datagen.flow_from_directory(
    directory=test_dir,
    target_size=(299, 299),
    color_mode="rgb",
    batch_size=64,
    class_mode="binary",
    shuffle=True,
    seed=42
)


Using TensorFlow backend.


Found 1461 images belonging to 2 classes.


In [2]:
from keras import optimizers
from keras import losses
from keras import metrics
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(32, (5, 5), border_mode='same', input_shape=(299,299,3),data_format="channels_last", activation='relu'))
model.add(Conv2D(32, (5, 5), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (5, 5), border_mode='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
    
model.compile(loss=losses.binary_crossentropy,
            optimizer=optimizers.RMSprop(lr=0.0001),
            metrics=[metrics.binary_accuracy])

model.load_weights('model8.4_10-0.21.hdf5')

C:\Users\TrappedIonPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(299, 299,..., data_format="channels_last", activation="relu", padding="same")`
  from ipykernel import kernelapp as app
C:\Users\TrappedIonPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
C:\Users\TrappedIonPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
C:\Users\TrappedIonPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
C:\Users\TrappedIonPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` c

In [3]:
results=model.evaluate_generator(test_generator, callbacks=None,  
                    use_multiprocessing=False, verbose=0)
print(results)

[0.42462214827537537, 0.8220397233963013]


In [4]:
from __future__ import print_function

import numpy as np
import cv2
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import jsonlines

def prepare_frames_from_video(datadir, fps = 30.0):
    video_list = []
    times = []
    frames = []
    if os.path.isdir(datadir):
        for root, dirs, files in os.walk(datadir):
            for file in files:
                if '.mp4' in file:
                    video_list.append(os.path.join(root, file))
    elif os.path.isfile(datadir):
        video_list = [datadir]
    else:
        print('Dataset is empty.')
    
    frames_index = 0
    frame_time = 0
    for video in video_list:
        cap = cv2.VideoCapture(video)
        count_index = 0
        print("start loading "+os.path.basename(video))
        t_start = time.time()
        while (True):
            success, frame = cap.read()
            if not success:
                break
            count_index += 1
            frame_time += 1
            if count_index == 10:
                frames.append(cv2.resize(frame,(299, 299))/255.)
                times.append(frame_time/float(fps))
                count_index = 0
                frames_index+=1
        print('loading data finished')
        t_end = time.time()
        print('time cost',t_end-t_start,'s')
        cap.release()
        cv2.destroyAllWindows()

    return frames, os.path.basename(video), times

In [5]:
import matplotlib.pyplot as plt
import jsonlines
import numpy as np
import cv2
from tqdm import tqdm
path = 'sample_video'
video_list = []
for root, dirs, files in os.walk(path):
    for file in files:
        if '.mp4' in file:
            frames, video, times = prepare_frames_from_video(os.path.join(root, file))
            prob = model.predict([frames])
            pred = [int(p>0.5) for p in prob]
            js = ""
            for i, p in enumerate(prob):
                js += "({},{}) ".format(times[i], p)
            with jsonlines.open(video.split('.')[0]+"_4.jsonl",mode='w') as writer:
                writer.write(js)
            plt.plot(times, pred)
            plt.xlabel('time (s)')
            plt.ylabel('prediction')
            plt.savefig(video.split('.')[0]+"_4.jpg")
            plt.clf()

start loading sample1.mp4
loading data finished
time cost 2.9992849826812744 s
start loading sample2.mp4
loading data finished
time cost 4.186527729034424 s
start loading sample3.mp4
loading data finished
time cost 5.904841423034668 s
start loading sample4.mp4
loading data finished
time cost 5.0144572257995605 s
start loading sample5.mp4
loading data finished
time cost 6.045464038848877 s


<Figure size 432x288 with 0 Axes>